<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip


--2025-08-09 17:14:32--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.134.84, 52.219.169.37, 3.5.139.177, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.134.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3103291675 (2.9G) [application/zip]
Saving to: ‘2011_10_03_drive_0047_sync.zip’

2011_10_03_drive_00 100%[===================>]   2.89G  14.0MB/s    in 1m 45s  

2025-08-09 17:16:19 (28.1 MB/s) - ‘2011_10_03_drive_0047_sync.zip’ saved [3103291675/3103291675]



In [2]:
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip

--2025-08-09 17:20:17--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.75.83, 52.219.46.91, 3.5.138.16, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.75.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4075 (4.0K) [application/zip]
Saving to: ‘2011_10_03_calib.zip’

2011_10_03_calib.zi 100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-08-09 17:20:17 (309 MB/s) - ‘2011_10_03_calib.zip’ saved [4075/4075]



In [3]:
!unzip -q '*.zip'


2 archives were successfully processed.


In [4]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import pandas as pd
import torch


In [5]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)

In [6]:
!wget https://github.com/itberrios/CV_tracking/raw/main/kitti_tracker/kitti_utils.py

--2025-08-09 17:27:35--  https://github.com/itberrios/CV_tracking/raw/main/kitti_tracker/kitti_utils.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/itberrios/CV_tracking/main/kitti_tracker/kitti_utils.py [following]
--2025-08-09 17:27:35--  https://raw.githubusercontent.com/itberrios/CV_tracking/main/kitti_tracker/kitti_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9759 (9.5K) [text/plain]
Saving to: ‘kitti_utils.py’

kitti_utils.py      100%[===================>]   9.53K  --.-KB/s    in 0s      

2025-08-09 17:27:35 (22.8 MB/s) - ‘kitti_utils.py’ saved [9759/9759]



In [7]:
DATA_PATH = r'2011_10_03/2011_10_03_drive_0047_sync'
# get RGB camera data
left_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_02/data/*png')))
right_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_03/data/*png')))

# getting LiDAR data
bin_paths = sorted(glob(os.path.join(DATA_PATH, 'velodyne_points/data/*bin')))

# Get GPS/IMU data
oxts_paths = sorted(glob(os.path.join(DATA_PATH, r'oxts/data/*txt')))

print(f'Number of RGB images: {len(left_image_paths)}')
print(f'Number of RGB images: {len(right_image_paths)}')
print(f'Number of LiDAR points: {len(bin_paths)}')
print(f'Number of GPS/IMU data: {len(oxts_paths)}')

Number of RGB images: 837
Number of RGB images: 837
Number of LiDAR points: 837
Number of GPS/IMU data: 837


In [ ]:
import mmcv
print(mmcv.__version__)  # Should print 2.1.0
import mmdet
import mmdet3d
print(mmdet.__version__, mmdet3d.__version__)  # Should print 3.3.0, 1.4.0

In [ ]:
from mmdet3d.datasets import build_dataset
cfg.data.train.data_root = './data/kitti/'
dataset = build_dataset(cfg.data.train)

In [ ]:

model = torch.hub.load('pytorch/vision:v0.15.2', 'deeplabv3_resnet101', pretrained=True)
model.eval()


In [ ]:
import torchvision.transforms as T
from PIL import Image
import numpy as np

transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406],
                                               std=[0.229, 0.224, 0.225])])

def segment_image(image_path):
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    sem_prob = torch.softmax(output, dim=0).cpu().numpy()  # shape: [21, H, W]
    return sem_prob


In [ ]:
import numpy as np

def project_lidar_to_image(points, calib):
    # points: Nx3 array (x,y,z)
    # calib: calibration matrix from KITTI
    # Return: Nx2 array of pixel coordinates
    pts_3d = np.hstack((points, np.ones((points.shape[0],1))))
    pts_2d = calib @ pts_3d.T
    pts_2d[:2, :] /= pts_2d[2, :]
    return pts_2d[:2, :].T  # Nx2 pixel coordinates